In [1]:
import pandas as pd
import datetime
import numpy as np
import os

d = '2022-09-30 00:00:00'
txn_dirPath = '/Users/rcheung/Documents/inv_mgmt_report/based_on_inv2019/txns/'


txn_converter = {'TXN - Transaction Type': str,
                 'TXN - Transaction Date': str,
                 'TXN - Item ID': str,
                 'TXN - Qty': float,
                 'TXN - Total Cost': float,
                 'TXN - Adjust Type': str}

#RC: add TXN-Sequence Nbr
transaction_col = ['TXN - Sequence Nbr', 'TXN - Transaction Type', 'TXN - Transaction Date',
                   'TXN - Item ID', 'TXN - Qty', 'TXN - Total Cost', 'TXN - Adjust Type']

out_types = ['051', '054,', '030', '031', '012']
positive_types = ['041', '022', '024', '050', '010', '020']
drop_types = ['053', '060']


# return all excel files in the directory as a list
def get_lof(directory):
    lof = []
    for file in os.listdir(directory):
        if file.endswith('.xlsx'):
            lof.append(directory + file)
    return lof


def read_txn_to_df(lof, converter):
    dataframe = pd.DataFrame()
    for f in lof:
        next_txn = pd.read_excel(f, converters=converter)
        #RC: changed to concat
        dataframe = pd.concat([dataframe, next_txn])
        print(f + ' is appended')
    return dataframe


# handle the increase/decrease column
def handle_I_D(txn_df):
    txn_df['TXN - Qty'] = np.where(txn_df['TXN - Adjust Type'] == 'D', 0 - abs(txn_df['TXN - Qty']),
                                   txn_df['TXN - Qty'])
    txn_df['TXN - Qty'] = np.where(txn_df['TXN - Adjust Type'] == 'M', 0 - abs(txn_df['TXN - Qty']),
                                   txn_df['TXN - Qty'])
    txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Adjust Type'] == 'D', 0 - abs(txn_df['TXN - Total Cost']),
                                          txn_df['TXN - Total Cost'])
    txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Adjust Type'] == 'M', 0 - abs(txn_df['TXN - Total Cost']),
                                          txn_df['TXN - Total Cost'])
    
    #RC: removed abs(), changed syntax
    #txn_df['TXN - Qty'] = np.where(txn_df['TXN - Transaction Type'].isin(out_types), 0 - txn_df['TXN - Qty'],
    #                               txn_df['TXN - Qty'])
    #txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Transaction Type'].isin(out_types), 0 - txn_df['TXN - Total Cost'],
    #                                      txn_df['TXN - Total Cost'])

    # Trust Anni's Code – it has already been validated.

    # We don't want to drop columns.
    #txn_df = txn_df.drop(columns=['TXN - Transaction Type', 'TXN - Adjust Type'])
    return txn_df

# eg: date = '2021-11-01 00:00:00'
# return all txn before 2021-11-01
def read_txn_by_date(txn_df, date):
    txn_df = txn_df.loc[txn_df['TXN - Transaction Date'] < date]
    #txn_df = txn_df.drop(columns=['TXN - Transaction Date'])
    # We don't want to drop the Transaction Date column.
    return txn_df


In [3]:
inv_converter = {'Report Date': str,
                 'Unit' : str,
                 'Item': str,
                 'Qty On Hand': float,
                 'On Hand Value': float}

invbybase_041119 = pd.read_excel("KJ_INV_BY_BASE_2019-04-11.xlsx", converters=inv_converter)

inv_starting_vals = invbybase_041119.groupby(['Item']).agg(
                        {'Qty On Hand': 'sum', 'On Hand Value': 'sum' }).reset_index()

inv_rename_dict = {'Item': 'TXN - Item ID', 'Qty On Hand': 'TXN - Overall Qty', 'On Hand Value': 'TXN - Overall Total Cost'}
inv_starting_vals.rename(columns = inv_rename_dict, inplace = True)

txn_columns = ['TXN - Item ID', 'TXN - Outgoing Qty','TXN - Outgoing Total Cost','TXN - Incoming Qty','TXN - Incoming Total Cost', 'TXN - Overall Qty', 'TXN - Overall Total Cost']
inv_starting_vals = inv_starting_vals.reindex(columns = txn_columns).fillna(0)

inv_starting_vals 


,TXN - Item ID,TXN - Outgoing Qty,TXN - Outgoing Total Cost,TXN - Incoming Qty,TXN - Incoming Total Cost,TXN - Overall Qty,TXN - Overall Total Cost
0,00118002,0.0,0.0,0.0,0.0,0.0,0.000
1,00118003,0.0,0.0,0.0,0.0,0.0,0.000
2,00118004,0.0,0.0,0.0,0.0,10.0,87.906
3,00118005,0.0,0.0,0.0,0.0,0.0,0.000
4,00118006,0.0,0.0,0.0,0.0,10.0,70.000
...,...,...,...,...,...,...,...
50248,YAV70M10,0.0,0.0,0.0,0.0,0.0,0.000
50249,YAV70M8,0.0,0.0,0.0,0.0,0.0,0.000
50250,YAV95M6,0.0,0.0,0.0,0.0,0.0,0.000
50251,Z53921,0.0,0.0,0.0,0.0,0.0,0.000


In [4]:
txn_df = read_txn_to_df(get_lof(txn_dirPath), txn_converter)

NameError: name 'pandas' is not defined

In [ ]:
# dataframe = txn_df.copy(deep = True)
#RC: switched order of filter and drop duplicate
# filter dataframe
dataframe = txn_df.loc[txn_df['TXN - Transaction Type'].isin(['010', '020', '012', '022', '024', '030', '031', '041', '050', '051', '054', '053', '060'])]

# drop duplication
dataframe = dataframe.drop_duplicates()

#keep relevant rows
dataframe = dataframe[transaction_col]

#dataframe = read_txn_by_date(dataframe,d)
dataframe = handle_I_D(dataframe)
dataframe

,TXN - Sequence Nbr,TXN - Transaction Type,TXN - Transaction Date,TXN - Item ID,TXN - Qty,TXN - Total Cost,TXN - Adjust Type
0,6,022,2019-04-11 00:00:00,04595358,3.0,77.37,NaN
1,2,022,2019-04-11 00:00:00,04282108,2.0,20.73,NaN
2,1,022,2019-04-11 00:00:00,02681140,1.0,45.00,NaN
3,5,022,2019-04-11 00:00:00,04595353,3.0,17.40,NaN
4,3,022,2019-04-11 00:00:00,04296009,2.0,74.59,NaN
...,...,...,...,...,...,...,...
9276,1,041,2022-10-05 00:00:00,03501351,0.0,0.00,NaN
9277,1,041,2022-10-05 00:00:00,02690605,0.0,0.00,NaN
9278,2,041,2022-10-05 00:00:00,02690605,0.0,0.00,NaN
9279,1,041,2022-10-05 00:00:00,02692249,0.0,0.00,NaN


In [ ]:
# calculate outs
df_outgoing = dataframe.loc[dataframe['TXN - Transaction Type'].isin(out_types)]
# group by date and item ID fields
df_outgoing = df_outgoing.groupby(['TXN - Item ID', 'TXN - Transaction Date']).agg(
                        {'TXN - Qty': 'sum', 'TXN - Total Cost': 'sum' }).reset_index()

# calculate ins
df_incoming = dataframe.loc[dataframe['TXN - Transaction Type'].isin(positive_types)]
# group by date and item ID fields
df_incoming = df_incoming.groupby(['TXN - Item ID', 'TXN - Transaction Date']).agg(
                        {'TXN - Qty': 'sum', 'TXN - Total Cost': 'sum' }).reset_index()

In [59]:
# combine ingoing and outgoing dfs by item and date
df_byitem = pd.merge(df_outgoing, df_incoming, on=['TXN - Item ID', 'TXN - Transaction Date'], how='outer')
df_byitem.update(df_byitem[['TXN - Qty_x', 'TXN - Total Cost_x', 'TXN - Qty_y',	'TXN - Total Cost_y']].fillna(0))

rename_dict = {'TXN - Qty_x':'TXN - Outgoing Qty', 'TXN - Total Cost_x':'TXN - Outgoing Total Cost', 'TXN - Qty_y':'TXN - Incoming Qty', 'TXN - Total Cost_y':'TXN - Incoming Total Cost'}

# call rename () method
df_byitem.rename(columns= rename_dict, inplace=True)

df_merged = pd.concat([df_byitem, inv_starting_vals])

df_merged = df_merged.sort_values(by = ['TXN - Item ID', 'TXN - Transaction Date'], ascending = [True, True], na_position = 'first').reset_index(drop=True)

In [60]:
df_merged

,TXN - Item ID,TXN - Transaction Date,TXN - Outgoing Qty,TXN - Outgoing Total Cost,TXN - Incoming Qty,TXN - Incoming Total Cost,TXN - Overall Qty,TXN - Overall Total Cost
0,00118002,NaN,0.0,0.0,0.0,0.0,0.0,0.000
1,00118003,NaN,0.0,0.0,0.0,0.0,0.0,0.000
2,00118004,NaN,0.0,0.0,0.0,0.0,10.0,87.906
3,00118004,2019-11-12 00:00:00,0.0,0.0,0.0,0.0,NaN,NaN
4,00118004,2020-10-27 00:00:00,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...
1383531,YAV70M10,NaN,0.0,0.0,0.0,0.0,0.0,0.000
1383532,YAV70M8,NaN,0.0,0.0,0.0,0.0,0.0,0.000
1383533,YAV95M6,NaN,0.0,0.0,0.0,0.0,0.0,0.000
1383534,Z53921,NaN,0.0,0.0,0.0,0.0,0.0,0.000


In [62]:
original_len = len(df_byitem)
for i in range(0, original_len):
    if i > 0 and df_byitem.loc[i, 'TXN - Item ID'] == df_byitem.loc[i-1, 'TXN - Item ID']:
        df_byitem.loc[i, 'TXN - Overall Qty'] = df_byitem.loc[i-1, 'TXN - Overall Qty'] + df_byitem.loc[i, 'TXN - Incoming Qty'] - df_byitem.loc[i, 'TXN - Outgoing Qty']
        df_byitem.loc[i, 'TXN - Overall Total Cost'] = df_byitem.loc[i-1, 'TXN - Overall Total Cost'] + df_byitem.loc[i, 'TXN - Incoming Total Cost'] - df_byitem.loc[i, 'TXN - Outgoing Total Cost']
    else:
        if df_byitem.loc[i, 'TXN - Item ID'] not in inv_starting_vals.index:
            df_byitem.loc[i, 'TXN - Overall Qty'] = df_byitem.loc[i, 'TXN - Incoming Qty'] - df_byitem.loc[i, 'TXN - Outgoing Qty']
            df_byitem.loc[i, 'TXN - Overall Total Cost'] = df_byitem.loc[i, 'TXN - Incoming Total Cost'] -  df_byitem.loc[i, 'TXN - Outgoing Total Cost']

In [ ]:
df_byitem

,TXN - Item ID,TXN - Transaction Date,TXN - Outgoing Qty,TXN - Outgoing Total Cost,TXN - Incoming Qty,TXN - Incoming Total Cost,TXN - Overall Qty,TXN - Overall Total Cost
0,00170009,2019-07-03 00:00:00,5.0,9.75,-35.0,-81.35,265.0,402.551
1,00170009,2019-08-12 00:00:00,40.0,78.00,0.0,0.00,225.0,324.551
2,00170009,2019-10-01 00:00:00,45.0,87.75,0.0,0.00,180.0,236.801
3,00170009,2020-01-09 00:00:00,50.0,97.50,50.0,97.50,180.0,236.801
4,00170009,2021-05-27 00:00:00,100.0,228.00,0.0,0.00,80.0,8.801
...,...,...,...,...,...,...,...,...
1333278,TD100800G01,2020-08-27 00:00:00,0.0,0.00,2.0,0.00,2.0,0.000
1333279,TD100808G01,2020-08-27 00:00:00,0.0,0.00,2.0,0.00,2.0,0.000
1333280,TD100809G01,2020-08-27 00:00:00,0.0,0.00,2.0,0.00,2.0,0.000
1333281,X1406001,2019-10-18 00:00:00,0.0,0.00,100.0,100.00,100.0,100.000


In [46]:
df_bydate = df_merged.groupby(['TXN - Transaction Date']).agg(
                        {'TXN - Outgoing Qty': 'sum', 'TXN - Outgoing Total Cost': 'sum', 'TXN - Incoming Qty': 'sum', 'TXN - Incoming Total Cost': 'sum'}).reset_index()

for i in range(0, len(df_bydate)):
    if i > 0:
        df_bydate.loc[i, 'TXN - Overall Qty'] = df_bydate.loc[i-1, 'TXN - Overall Qty'] + df_bydate.loc[i, 'TXN - Incoming Qty'] + df_bydate.loc[i, 'TXN - Outgoing Qty']
        df_bydate.loc[i, 'TXN - Overall Total Cost'] = df_bydate.loc[i-1, 'TXN - Overall Total Cost'] + df_bydate.loc[i, 'TXN - Incoming Total Cost'] + df_bydate.loc[i, 'TXN - Outgoing Total Cost']
    else:
        df_bydate.loc[i, 'TXN - Overall Qty'] = df_bydate.loc[i, 'TXN - Incoming Qty'] + df_bydate.loc[i, 'TXN - Outgoing Qty']
        df_bydate.loc[i, 'TXN - Overall Total Cost'] = df_bydate.loc[i, 'TXN - Incoming Total Cost'] + df_bydate.loc[i, 'TXN - Outgoing Total Cost']

In [47]:
df_bydate

,TXN - Transaction Date,TXN - Outgoing Qty,TXN - Outgoing Total Cost,TXN - Incoming Qty,TXN - Incoming Total Cost,TXN - Overall Qty,TXN - Overall Total Cost
0,2019-04-11 00:00:00,11623.0,219494.61,22974.0,506982.33,34597.00,7.264769e+05
1,2019-04-12 00:00:00,6874.0,135647.22,8904.0,189454.30,50375.00,1.051578e+06
2,2019-04-13 00:00:00,682.0,20545.41,12.0,53.83,51069.00,1.072178e+06
3,2019-04-14 00:00:00,545.0,9654.32,0.0,0.00,51614.00,1.081832e+06
4,2019-04-15 00:00:00,6613.0,192705.50,7084.0,165456.90,65311.00,1.439994e+06
...,...,...,...,...,...,...,...
1258,2022-10-01 00:00:00,821.0,24613.00,0.0,0.00,26198935.57,3.735680e+08
1259,2022-10-02 00:00:00,1224.0,28814.91,0.0,0.00,26200159.57,3.735968e+08
1260,2022-10-03 00:00:00,11934.0,153810.30,14860.0,172298.13,26226953.57,3.739229e+08
1261,2022-10-04 00:00:00,9691.0,193010.00,8718.0,286804.83,26245362.57,3.744028e+08


In [48]:
df_byitem.to_csv('output/txns_by_item.csv', index = None)
df_bydate.to_csv('output/txns_by_date.csv', index = None)

In [43]:
inv_starting_vals.to_csv('output/starting_vals.csv')